In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import dotenv

In [ ]:

from three_stages_llm import ThreeStagesGraphGenerator
from three_stages_algo import ThreeStagesGraphGenerator as AlgoGenerator

In [4]:
from dialogue2graph.pipelines.core.graph import Graph
from dialogue2graph.pipelines.core.dialogue_sampling import RecursiveDialogueSampler
from dialogue2graph.datasets.complex_dialogues.prompts import graph_example
from dialogue2graph.metrics.llm_metrics import compare_graphs
from dialogue2graph.metrics.no_llm_metrics import match_dg_triplets

In [10]:
from dialogue2graph.datasets.complex_dialogues.generation import LoopedGraphGenerator

In [5]:
from dialogue2graph.pipelines.model_storage import ModelStorage

In [8]:
dotenv.load_dotenv()

True

In [ ]:
Graph(graph_example).visualise_short("graph_example")

In [ ]:
graph_example["nodes"]

In [7]:
sampler = RecursiveDialogueSampler()

In [ ]:
dias = sampler.invoke(Graph(graph_example), 5)

In [ ]:
ms = ModelStorage()
ms.add(
        key="grouping_llm",
        config={"model_name": "chatgpt-4o-latest", "temperature": 0},
        model_type="llm",
    )    
ms.add(
        key="extending_llm",
        config={"model_name": "chatgpt-4o-latest", "temperature": 0},
        model_type="llm",
    )    
ms.add(
        key="filling_llm",
        config={"model_name": "o3-mini", "temperature": 1},
        model_type="llm",
    )
ms.add(
        key="formatting_llm",
        config={"model_name": "gpt-4o-mini", "temperature": 0},
        model_type="llm",
    )
ms.add(
        key="sim_model",
        config={"model_name": "BAAI/bge-m3", "device": "cuda:0"},
        model_type="emb",
    )


In [11]:
ms.add(
        key="val_llm",
        config={"model_name": "gpt-3.5-turbo"},
        model_type="llm",
    )

INFO:dialogue2graph.pipelines.model_storage:Added llm model 'val_llm' to storage


In [15]:
topic_generator = LoopedGraphGenerator(
    ms, "grouping_llm", "val_llm", "grouping_llm", "formatting_llm"
)

In [ ]:
graph = topic_generator.invoke("booking a table")

In [ ]:
graph = topic_generator.invoke("arranging airport transfers")

In [ ]:
Graph(graph[0]["graph"]).visualise_short("transfer")

In [15]:
import json

In [33]:
with open("transfer.json", "w", encoding="utf-8") as f:
    json.dump(graph[0], f, ensure_ascii=False, indent=4)

In [10]:
graph_c = Graph(
    {
        "edges": [
            {
                "source": 1,
                "target": 2,
                "utterances": [
                    "I’d like to book a table for dinner.",
                    "Can you help me reserve a table at a restaurant?",
                ],
            },
            {
                "source": 2,
                "target": 3,
                "utterances": ["Tomorrow at 7 PM.", "This Saturday at 8 PM."],
            },
            {
                "source": 3,
                "target": 4,
                "utterances": ["For four people.", "A table for two, please."],
            },
            {
                "source": 4,
                "target": 8,
                "utterances": [
                    "Somewhere in downtown would be nice.",
                    "Anywhere with good Italian food.",
                ],
            },
            {
                "source": 8,
                "target": 9,
                "utterances": ["Let's go with Italian.", "Chinese sounds good."],
            },
            {
                "source": 9,
                "target": 13,
                "utterances": [
                    "Bella Italia sounds perfect.",
                    "I’ll go with Lotus Garden.",
                ],
            },
            {
                "source": 13,
                "target": 5,
                "utterances": ["Yes, that works.", "That sounds great."],
            },
            {
                "source": 5,
                "target": 10,
                "utterances": ["Yes, that’s correct.", "Everything looks good."],
            },
            {
                "source": 5,
                "target": 6,
                "utterances": [
                    "Actually, can we change the time to 9 PM?",
                    "Let's do a different date instead.",
                ],
            },
            {
                "source": 5,
                "target": 7,
                "utterances": [
                    "Wait, I need to change the number of people.",
                    "Can we make it for six instead?",
                ],
            },
            {
                "source": 6,
                "target": 5,
                "utterances": ["Make it 9 PM instead.", "Let's go for Sunday at 7 PM."],
            },
            {
                "source": 7,
                "target": 5,
                "utterances": [
                    "Now it's for three people.",
                    "Actually, let's do five.",
                ],
            },
            {
                "source": 10,
                "target": 11,
                "utterances": [
                    "Can I get their phone number and address?",
                    "Do you have their contact info?",
                ],
            },
            {
                "source": 10,
                "target": 12,
                "utterances": [
                    "Nope, that's all. Thanks!",
                    "That’s all I needed. Have a great day!",
                ],
            },
            {
                "source": 11,
                "target": 12,
                "utterances": [
                    "No, that's all. Thanks!",
                    "I’m all set. Have a great day!",
                ],
            },
        ],
        "nodes": [
            {
                "id": 1,
                "label": "start",
                "is_start": True,
                "utterances": ["Hey there! How can I help you today?"],
            },
            {
                "id": 2,
                "label": "ask_date_time",
                "is_start": False,
                "utterances": [
                    "Sure! What date and time would you like to book the table for?"
                ],
            },
            {
                "id": 3,
                "label": "ask_party_size",
                "is_start": False,
                "utterances": ["Got it. How many people will be joining?"],
            },
            {
                "id": 4,
                "label": "ask_location",
                "is_start": False,
                "utterances": [
                    "Alright! Do you have a specific location in mind or any preferences?"
                ],
            },
            {
                "id": 5,
                "label": "confirm_details",
                "is_start": False,
                "utterances": [
                    "Just to confirm, you need a table for {party_size} on {date_time}. Does that sound right?"
                ],
            },
            {
                "id": 6,
                "label": "modification_date_time",
                "is_start": False,
                "utterances": ["Sure! What new date and time would you like instead?"],
            },
            {
                "id": 7,
                "label": "modification_party_size",
                "is_start": False,
                "utterances": ["Got it! How many people should I update it to?"],
            },
            {
                "id": 8,
                "label": "offer_options",
                "is_start": False,
                "utterances": [
                    "There are a few great restaurants available. Would you like Italian, Chinese, or something else?"
                ],
            },
            {
                "id": 9,
                "label": "provide_options",
                "is_start": False,
                "utterances": [
                    "Sure! You could try Bella Italia for Italian, Lotus Garden for Chinese, or Grill House for a steakhouse experience. Any of these sound good?"
                ],
            },
            {
                "id": 10,
                "label": "confirm_booking",
                "is_start": False,
                "utterances": [
                    "Great, I've booked your table at {restaurant_name} for {party_size} on {date_time}. Anything else I can help with?"
                ],
            },
            {
                "id": 11,
                "label": "provide_contact_info",
                "is_start": False,
                "utterances": [
                    "Bella Italia is located at 25 Main Street and you can call them at 555-1234.",
                    "Grill House is on West Road, and they can be reached at 555-8765.",
                    "Lotus Garden is at 78 East Avenue and their phone number is 555-5678.",
                ],
            },
            {
                "id": 12,
                "label": "closing",
                "is_start": False,
                "utterances": [
                    "Thanks for reaching out. Have a great day!",
                    "You're all set! Enjoy your meal!",
                    "Your reservation is confirmed. Hope you have a wonderful time!",
                ],
            },
            {
                "id": 13,
                "label": "confirm_restaurant",
                "is_start": False,
                "utterances": [
                    "Great choice! You’d like to book a table at {restaurant_name}, correct?"
                ],
            },
        ],
    }
)

In [11]:
dias = sampler.invoke(graph_c, 5)

Graph has all the dialogues


In [ ]:
graph_c.visualise_short("Graph")

In [ ]:
graph_c.graph_dict["nodes"]

In [ ]:
dias = sampler.invoke(graph_c, 5)

In [39]:
generator = ThreeStagesGraphGenerator()

In [ ]:
res_graph = generator.invoke(dias, temp=1)

In [ ]:
res_graph = generator.invoke(dias, model_name="gpt-4o-2024-11-20", temp=0)

In [ ]:
res_graph = generator.invoke(dias, model_name="gpt-4o", temp=0)

In [ ]:
res_graph = generator.invoke(dias, model_name="o1-mini", temp=1)

In [ ]:
graph_example

In [ ]:
res_graph.graph_dict

In [ ]:
res_graph.visualise_short("Generated from graph dialogues")

In [ ]:
compare_graphs(res_graph, graph_c)

In [35]:
algo_generator = AlgoGenerator()

In [37]:
from dialogue2graph.pipelines.core.dialogue import Dialogue

In [38]:
dialogues = [Dialogue().from_list(seq["messages"]) for seq in graph[0]["dialogues"]]

In [ ]:
res_graph = algo_generator.invoke(dialogues, model_name="gpt-4o", temp=0)

In [ ]:
res_graph.visualise_short("Generated from graph dialogues")

In [ ]:
compare_graphs(res_graph, Graph(graph[0]["graph"]))

In [ ]:
graph = topic_generator.invoke("booking a cruise package")

In [ ]:
graph[0]["graph"]["nodes"]

In [ ]:
graph[0]["graph"]["edges"]

In [ ]:
Graph(graph[0]["graph"]).visualise_short("Booking a cruise")

In [ ]:
dias = sampler.invoke(Graph(graph[0]["graph"]), 5)

In [ ]:
res_graph = generator.invoke(dias)

In [ ]:
res_graph.visualise_short("Booking a Cruise from dialogues")

In [ ]:
compare_graphs(res_graph, Graph(graph[0]["graph"]))

In [ ]:
res_graph = algo_generator.invoke(dias, model_name="gpt-4o", temp=0)

In [ ]:
res_graph.visualise_short("Booking a Cruise from dialogues")

In [ ]:
compare_graphs(res_graph, Graph(graph[0]["graph"]))

In [ ]:
graph = topic_generator.invoke("passport renewal application")

In [ ]:
graph

In [ ]:
Graph(graph[0]["graph"]).visualise_short("Passport renewal")

In [ ]:
graph[0]["graph"]["nodes"]

In [ ]:
dias = sampler.invoke(Graph(graph[0]["graph"]), 5)

In [ ]:
res_graph = algo_generator.invoke(dias, model_name="gpt-4o", temp=0)

In [ ]:
compare_graphs(res_graph, Graph(graph[0]["graph"]))

In [ ]:
match_dg_triplets(Graph(graph_c), dias)

In [ ]:
dias = sampler.invoke(Graph(graph_example), 5)

In [ ]:
match_dg_triplets(Graph(graph_c), dias)

In [ ]:
from dialogue2graph.metrics.no_llm_metrics import are_valid_paths

In [ ]:
are_valid_paths(Graph(graph_c), dias)

In [ ]:
dias = sampler.invoke(Graph(graph_example), 5)

In [ ]:
are_valid_paths(Graph(graph_c), dias)